In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
import sys
import os
import pickle
import random

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from torch.optim import Adam
from sklearn import metrics
from collections import OrderedDict 

sys.path.append(os.path.abspath('..'))

import datasets
import utils.more_torch_functions as mtf
from utils.modules import Parallel, MaxLayer
from utils.custom_activations import StepActivation
from utils.custom_loss import AsymBCELoss
from utils.misc import train_model

seed = 2872
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

SAVE_PATH = os.path.join(os.path.abspath('..'), "backup")
PKL_PATH = os.path.join(SAVE_PATH, "bdd")
PTH_PATH = os.path.join(SAVE_PATH, "nn")
METRIC_PATH = os.path.join(SAVE_PATH, "metrics")

## Load data

In [3]:
dataset = datasets.DiabetesDataset

np_x, np_y = dataset.get_dataset(balancing=True, discretizing=True, hot_encoding=True)
x_data, y_data = torch.Tensor(np_x), torch.Tensor(np_y)
input_size = x_data.size(1)
print(x_data.size())

torch.Size([536, 29])


In [4]:
def eval_model(x_train, y_train, x_valid, y_valid, model, metrics_average="binary"):
    model.eval()
    pred_train = model(x_train).detach()
    pred_valid = model(x_valid).detach()

    p_train, r_train, f_train, _ = metrics.precision_recall_fscore_support(y_train, pred_train, beta=1, average=metrics_average, labels=[0,1])
    p_valid, r_valid, f_valid, _ = metrics.precision_recall_fscore_support(y_valid, pred_valid, beta=1, average=metrics_average, labels=[0,1])

    return f_train, p_train, r_train, f_valid, p_valid, r_valid

def train_eval_model(x_train, y_train, x_valid, y_valid, model, criterion, optimizer, epochs=50, metrics_average="binary"):
    train_model(x_train, y_train, model, criterion, optimizer, epochs)
    return eval_model(x_train, y_train, x_valid, y_valid, model, metrics_average)

def print_eval(x_train, y_train, x_valid, y_valid, model, criterion, optimizer):
    f_train, p_train, r_train, f_valid, p_valid, r_valid = train_eval_model(x_train, y_train, x_valid, y_valid, model, criterion, optimizer)
    print(
        f"{'':<15}{'Train':^15}{'Valid':^15}",
        f"{'F1 score':<15}{f_train:^15.3f}{f_valid:^15.3f}",
        f"{'Precision':<15}{p_train:^15.3f}{p_valid:^15.3f}",
        f"{'Rappel':<15}{r_train:^15.3f}{r_valid:^15.3f}",
        sep="\n",
    )

## Network

In [5]:
class nApxNet(nn.Module):
    def __init__(self, n, hl=3, max_weighting="all") -> None:
        super().__init__()

        self.n_apx = n
        self.net = nn.Sequential(OrderedDict([
            ('nets', Parallel(OrderedDict([
                (f'apx{i}', ApproxNet(hl)) for i in range(1, self.n_apx+1)
            ]))),
            ('or_', MaxLayer(backward_method=max_weighting)),
        ]))

    def forward(self, input):
        return self.net(input)
    
    def forward_apx_only(self, input):
        return mtf.maximum([self.net.nets.get_submodule(f"apx{i}")(input) for i in range(1, self.n_apx+1)])
    
    def add_apx(self, module):
        self.n_apx += 1
        self.net.nets.add_module(f'apx{self.n_apx}', module)
    
    def add_nn(self, module):
        self.net.nets.add_module(f'nn', module)
    
class ApproxNet(nn.Module):
    def __init__(self, hl1):
        super().__init__()
        self.nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', StepActivation()),
            ('l2', nn.Linear(hl1,5)),
            ('a2', StepActivation()),
            ('l3', nn.Linear(5,1)),
            ('a3', StepActivation()),
        ]))        

    def forward(self, x):
        x = self.nn(x)

        return x
    
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        hl1 = 50
        hl2 = 25

        self. nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', nn.ReLU()),
            ('l2', nn.Linear(hl1,hl2)),
            ('a2', nn.ReLU()),
            ('l3', nn.Linear(hl2,1)),
            ('a3', StepActivation()),
        ]))
    
    def forward(self, x):
        x = self.nn(x)

        return x

## Exp

In [6]:
def compute_metrics(model, x_valid, y_valid):
    model.eval()
    global_pred_valid = model(x_valid).detach()
    apx_pred_valid = model.forward_apx_only(x_valid).detach()
    nn_pred_valid = model.net.nets.nn(x_valid).detach()

    glob_recall = metrics.recall_score(y_valid, global_pred_valid, pos_label=1, average='binary', zero_division=0)
    apx_recall = metrics.recall_score(y_valid, apx_pred_valid, pos_label=1, average='binary', zero_division=0)
    apx_misszero = 1 - metrics.precision_score(y_valid, apx_pred_valid, pos_label=1, average='binary', zero_division=0)
    nn_recall = metrics.recall_score(y_valid, nn_pred_valid, pos_label=1, average='binary', zero_division=0)
    relative_recall = apx_recall/glob_recall

    glob_prec = metrics.precision_score(y_valid, global_pred_valid, average='micro', zero_division=0)

    return glob_recall, apx_recall, apx_misszero, nn_recall, relative_recall, glob_prec

def sequential_train(x_train, y_train, x_valid, y_valid, epochs=500, n=1, max_weighting="all"):   
    model = nApxNet(0, hl=5, max_weighting=max_weighting)
    criterion = AsymBCELoss(2) # HERE
    for _ in range(n):
        module = ApproxNet(5)
        optimizer = Adam(module.parameters(), lr=1e-2, weight_decay=1e-6)
        model.add_apx(module)

        train_model(x_train, y_train, model, criterion, optimizer, epochs)
        mtf.freeze_model(model)

    module = Net()
    model.add_nn(module)
    criterion = nn.BCELoss() # CHANGE ???
    optimizer = Adam(module.parameters(), lr=1e-2, weight_decay=1e-6)

    train_model(x_train, y_train, model, criterion, optimizer, epochs)

    return compute_metrics(model, x_valid, y_valid)

def simultaneous_train(x_train, y_train, x_valid, y_valid, epochs=500, n=1, max_weighting="all"):    
    model = nApxNet(n, hl=5, max_weighting=max_weighting)
    criterion = AsymBCELoss(2) # HERE
    model.add_nn(Net())
    optimizer = Adam(model.parameters(), lr=1e-2, weight_decay=1e-6)

    train_model(x_train, y_train, model, criterion, optimizer, epochs)

    return compute_metrics(model, x_valid, y_valid)

In [7]:
results = dict()
max_iter = 100
for wieghting_method in ["all", "max"]:
    for func_train in [sequential_train, simultaneous_train]:
        sum = np.zeros(6) 
        for _ in range(max_iter):
            train_index, valid_index = torch.utils.data.random_split(range(x_data.size(0)), [0.9, 0.1])

            x_train, y_train = x_data[train_index], y_data[train_index]
            x_valid, y_valid = x_data[valid_index], y_data[valid_index]

            res = func_train(x_train, y_train, x_valid, y_valid, n=3, max_weighting=wieghting_method)
            res = np.array(res)
            sum+=res

        sum/=max_iter
        print(func_train.__name__.upper(), wieghting_method.upper())
        results[(func_train.__name__.upper(), wieghting_method.upper())] = sum.copy()
        print(f"Model cov : {sum[0]:.3f}",
            f"NN cov : {sum[3]:.3f}",
            f"Approx cov : {sum[1]:.3f}",
            f"Approx misclassified zeros : {sum[2]:.3f}", # number of zeros predicted as one / number predicted as one
            f"Relative cov : {sum[4]:.3f}",
            f"Model precision : {sum[5]:.3f}",
            sep='\n', end="\n\n") 

SEQUENTIAL_TRAIN ALL
Model cov : 0.808
NN cov : 0.631
Approx cov : 0.719
Approx misclassified zeros : 0.272
Relative cov : 0.888
Model precision : 0.728

SIMULTANEOUS_TRAIN ALL
Model cov : 0.812
NN cov : 0.341
Approx cov : 0.734
Approx misclassified zeros : 0.339
Relative cov : 0.892
Model precision : 0.727

SEQUENTIAL_TRAIN MAX
Model cov : 0.659
NN cov : 0.000
Approx cov : 0.659
Approx misclassified zeros : 0.262
Relative cov : 1.000
Model precision : 0.701

SIMULTANEOUS_TRAIN MAX
Model cov : 0.696
NN cov : 0.495
Approx cov : 0.236
Approx misclassified zeros : 0.560
Relative cov : 0.339
Model precision : 0.713



In [20]:
# 0: model cov, 1: apx cov, 2: apx miss, 3: nn cov, 4: relative cov, 5: model precision
print(results)
rbis = {k: list(v) for k, v in results.items()}
a, b, c, d = list(rbis.values())
values = [a, c, b, d]
for x in zip(*values):
    print(" & ".join([f"{a:.2f}" for a in x]))

{('SEQUENTIAL_TRAIN', 'ALL'): array([0.80788612, 0.71884743, 0.27201542, 0.63146051, 0.88773641,
       0.72754717]), ('SIMULTANEOUS_TRAIN', 'ALL'): array([0.81235637, 0.73426092, 0.33901588, 0.34148472, 0.89210092,
       0.72698113]), ('SEQUENTIAL_TRAIN', 'MAX'): array([0.65866333, 0.65866333, 0.26226979, 0.        , 1.        ,
       0.70132075]), ('SIMULTANEOUS_TRAIN', 'MAX'): array([0.69636098, 0.23619374, 0.55980637, 0.49500196, 0.3392874 ,
       0.71339623])}
0.81 & 0.66 & 0.81 & 0.70
0.72 & 0.66 & 0.73 & 0.24
0.27 & 0.26 & 0.34 & 0.56
0.63 & 0.00 & 0.34 & 0.50
0.89 & 1.00 & 0.89 & 0.34
0.73 & 0.70 & 0.73 & 0.71
